발표와 질의응답 : 발표를 누가 하든지 질의응답에서 대답을 잘 못하면 개별 감점이 있습니다.

보강이 필요한데, 영상으로 진행할 예정입니다. 세 시간 정도 앞에서 배운 내용을 요약해주는 형식입니다. 왠만해서는 영상으로 보강이 가능하도록 할 겁니다.

퀴즈는 발표주차 바로 전에 보도록 하겠습니다. 15~20분 정도 분량입니다. 간단히 확인할 수 있는 정도의 문제 -> 조별 편차가 얼마 안나기 때문에 만들어놓은 겁니다.

### SQLite

오픈 소스, 가벼운 유형으로 모든 시스템에서 사용이 가능하다.

데이터 형식의 다양성(스키마 지정), 수평 확장성(데이터베이스 네트워크 간 확장), 데이터 관리의 엄격성에서는 다소 약점이 있고 이런 부분에서는 MySQL이 강점을 가지고 있다.

> 실제 상황과의 괴리 : 실제는 서버에서 돌아가고, 대용량 데이터가 여러 데이터베이스간 교차하여 작동함.

지원하는 데이터 유형

* TEXT 문자열(UTF-8, UTF-16BE, UTF16LE)
* REAL 8바이트 부동 소수점 값. float32
* NULL
* INTEGER 8바이트 정수값. int32
* BLOB 입력형태 그대로 저장(이미지, 오디오, 멀티미디어 파일)

In [1]:
install.packages("DBI")
install.packages("RSQLite")
library(DBI)
library(RSQLite)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



### practice

In [85]:
library(DBI)
library(RSQLite)

In [92]:
data("mtcars")
mtcars$car_names = rownames(mtcars) ## 인덱스를 데이터로 따로 받아 새 열로 추가
rownames(mtcars) = NULL  ## 인덱스 초기화
head(mtcars)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,car_names
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4,Mazda RX4
2,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4,Mazda RX4 Wag
3,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1,Datsun 710
4,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1,Hornet 4 Drive
5,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2,Hornet Sportabout
6,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1,Valiant


`-` Creating Database File : `dbConnect()`

In [93]:
conn = dbConnect(RSQLite::SQLite(), "CarsDB.db")  ## create db file

`-` Writing table : `dbWriteTable()` 데이터프레임 삽입에 유리(append)

In [94]:
dbWriteTable(conn, "cars_data", mtcars)  ## Write table(relation)
dbListTables(conn)  ## debuging

[1] "cars_data"

In [95]:
car <- c('Camaro', 'California', 'Mustang', 'Explorer')
make <- c('Chevrolet','Ferrari','Ford','Ford')
df1 <- data.frame(car,make)
car <- c('Corolla', 'Lancer', 'Sportage', 'XE')
make <- c('Toyota','Mitsubishi','Kia','Jaguar')
df2 <- data.frame(car,make)

In [96]:
dfList = list(df1, df2)

for (i in 1:length(dfList)) {
    dbWriteTable(conn, "Cars_and_Makes", dfList[[i]], append = TRUE)  ## write new table
}

dbListTables(conn)

[1] "Cars_and_Makes" "cars_data"

* arguments

> append = TRUE : 기존 테이블에 추가
>
> 작성 후 반드시 dbDisconnect()를 하라고 경고가 발생

`-` Query : `dbGetQuery()` 결과물만 반환할 수 있음.

In [98]:
# dbGetQuery(conn, "SELECT * FROM Cars_and_Makes")
# dbGetQuery(conn, "SELECT * FROM cars_data LIMIT 20")
dbGetQuery(conn, "SELECT car_names, hp, cyl FROM cars_data")

car_names,hp,cyl
<chr>,<dbl>,<dbl>
Mazda RX4,110,6
Mazda RX4 Wag,110,6
Datsun 710,93,4
Hornet 4 Drive,110,6
Hornet Sportabout,175,8
Valiant,105,6
Duster 360,245,8
Merc 240D,62,4
Merc 230,95,4


`-` Query : `dbExecute()` 작업만 수행하고 결과물을 반환하지 않음. 스키마 생성 시 유용

In [99]:
conn = dbConnect(SQLite(), ".temp.db")

In [100]:
## creating table with schema
dbExecute(conn, "CREATE TABLE mytable
          (a INTEGER PRIMARY KEY, 
         b TEXT)")

[1] 0

In [101]:
dbExecute(conn, "INSERT INTO mytable VALUES(1, 'test'), (2, 'test'), (6, 'test')")

[1] 3

In [103]:
dbGetQuery(conn, "SELECT * FROM mytable")

a,b
<int>,<chr>
1,test
2,test
6,test


`-` Query : `dbSendQuery()` 작업을 수행하고 어떤 데이터가 들어갔는지 결과를 반환

In [105]:
dbSendQuery(conn, "INSERT INTO mytable VALUES(4, 'test')")

<SQLiteResult>
  SQL  INSERT INTO mytable VALUES(4, 'test')
  ROWS Fetched: 0 [complete]
       Changed: 1

In [106]:
dbGetQuery(conn, "SELECT * FROM mytable")

Warning message:
“Closing open result set, pending rows”


a,b
<int>,<chr>
1,test
2,test
4,test
6,test


`-` Removing table : `dbRemoveTable()`

In [ ]:
dbRemoveTable(conn, 'test_table')

`-` Query 문법

`"SELECT col_name1 AS col1, AVG(col2) AS average_col_2 FROM table GROUP BY col1 ORDER BY average_col_2"`

> 범용적인 문법, 테이블을 바꾸지 않음. `dbGetQuery()`에서 주로 사용.
>
> `AS`로 열 이름을 변경해서 호출, `AVG()`로 평균을 호출, `GROUP BY`에 해당하는 열 그룹을 기준으로 그룹화하여 통계량 산출, `ORDER BY`에 해당하는 열 그룹을 기준으로 sorting.

`"INSERT INTO table (col1, col2, ...) VALUES (val11, val12, ...), (val21, val22, ...), ..."`

> 테이블에 튜플 삽입, `dbExecute()`, `dbSendQuery()`에서 사용.

`"DELETE FROM table WHERE col_name = value"`

> 테이블의 튜플 제거, `dbExecute()`, `dbSendQuery()`에서 사용.

`"UPDATE table SET col_name = value WHERE col_name = value"`

> 테이블의 튜플 수정, `dbExecute()`, `dbSendQuery()`에서 사용.